In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset

import torch.utils.data as data

import numpy as np

import wandb

from tqdm import tqdm

import os

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Get data

In [2]:
###

## Dataset

In [23]:
import torch
from torch.utils.data import Dataset
import os

class ImageDataset(Dataset):
    def __init__(self, list_paths=None, transform=None, probs=None):
        """
        Args:
        - `list_paths`: List of paths to .pt files
        - `transform`: List of transformations to apply
        - `probs`: List of probabilities corresponding to each transformation
        """
        self.data = list_paths
        self.length = len(list_paths)
        self.transform = transform
        self.probs = probs

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        im_path = os.path.abspath(self.data[idx][0])

        # Load the .pt file and convert it to a PyTorch tensor
        im = torch.load(im_path)
        im = im.unsqueeze(0)  # Add a singleton dimension for channels

        y = self.data[idx][1]

        if self.transform:
            for i in range(len(self.transform)):
                rand_val = np.random.random()

                if rand_val <= self.probs[i]:
                    im = self.transform[i](im)

                    return im, y, idx

        return im, y, idx



In [24]:
# Transformations to be done on the images

pHflip = 0.2                                     # probability of applying a horizontal flip
pVflip = 0.2                                     # probability of applying a vertical flip
pInvert = 0.4
deg = 20


tr_base = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)),           # modify those values
])

trans1 = transforms.Compose([
    transforms.RandomHorizontalFlip(pHflip),
    transforms.RandomInvert(pInvert),
    transforms.RandomRotation(deg),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

trans_lits = [trans1, tr_base]

In [25]:
# Train and val folders
data_learning = "../../Data/input_tensors/train"
data_validation = "../../Data/input_tensors/val"

# Getting file paths for each .pt file in the folder
image_list_learning = [os.path.join(data_learning, pt_file) for pt_file in os.listdir(data_learning) if pt_file.endswith('.pt')]
image_list_validation = [os.path.join(data_validation, pt_file) for pt_file in os.listdir(data_validation) if pt_file.endswith('.pt')]

#loading datasets
dataset_learning = ImageDataset(list_paths=image_list_learning, transform=trans_lits, probs=[0.5, 0.5])
dataset_validation = ImageDataset(list_paths=image_list_validation, transform=[tr_base], probs=[1])

#loading dataloaders
loaderLearning = data.DataLoader(dataset_learning, batch_size = 5, shuffle= True, num_workers = 0)
loaderValidation = data.DataLoader(dataset_validation, batch_size = 5, shuffle= True, num_workers = 0)

In [27]:
#testing to see if it works
images, labels, ids = next(iter(loaderLearning))
print(images.shape)

PermissionError: [Errno 13] Permission denied: 'c:\\Users\\turla\\Documents\\4. Uliege\\Semester 3\\Computer Vision\\Cell-detection-and-counting-Algorithms\\Jerome test zone\\pipeline'

### TEST - Using simulated data

In [8]:
class imageDataset(Dataset):
    """
    Simple dataset. Supposed to load images and the corresponding output
    """

    def __init__(self, listPaths=list, transform=None, probs= None):
        """ 
        Args:
        -----
        - `listPaths`: list of images + number of cells [image, y]
        """
        self.data = torch.rand((20, 3, 400, 400))
        self.y = torch.rand((20, 1))
        self.length = self.data.shape[0]
        self.transform = transform
        self.probs = probs

    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        
        im = self.data[idx, :, :, :]
        y = self.y[idx]
        
        if self.transform:
            for i in range(len(self.transform)):
                randVal = np.random.random()
                
                if randVal <= self.probs[i]:
                    im = self.transfomr[i](im)
                    
                    return im, y, idx
        
        return im, y, idx

In [9]:
dataLearning = None
dataValidation = None
dataTest = None

datasetLearning = imageDataset(listPaths=None, transform=None, probs = [0.5, 0.5])
datasetValidation = imageDataset(listPaths=None, transform=None, probs = [1])            # no augmentation for the validation or test set
datasetTest = imageDataset(listPaths=None, transform=None, probs = [1])


loaderLearning = data.DataLoader(datasetLearning, batch_size = 100, shuffle= True, num_workers = 4)
loaderValidation = data.DataLoader(datasetValidation, batch_size = 100, shuffle= True, num_workers = 4)
loaderTest = data.DataLoader(datasetTest, batch_size = 100, shuffle= True, num_workers = 4)

In [10]:
images, labels, ids = next(iter(loaderLearning))
print(images.shape)

RuntimeError: DataLoader worker (pid(s) 6316, 17144, 10480, 20592) exited unexpectedly

### Model

In [6]:
model = models.resnet152(weights="IMAGENET1K_V2")
for param in model.parameters():
    param.requires_grad = False

In [7]:
mlp = nn.Sequential(
    nn.Linear(model.fc.in_features, 512),
    nn.ReLU(),
    nn.Linear(512, 512),
    nn.ReLU(),
    nn.Linear(512, 1)
)

model.fc = mlp

In [31]:
model = model.to(device)

### Learning

In [40]:
x_test = torch.rand((1, 3,700, 700)).to(device)

In [41]:
print(model(x_test))

tensor([[-0.0282]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [10]:
nbEpoch=40
loaderLearning = loaderLearning
loaderValidation = loaderValidation
loaderTest = loaderTest

criterion = nn.MSELoss()

In [32]:
wandb.init(project = 'master_thesis', name = "CV_init")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jepi1202 (uliege_action_spotting_2022_2023_context). Use `wandb login --relogin` to force relogin


In [33]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
wandb.watch(model, log = 'all', log_freq=100)
model.train()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [51]:
j = 0

for i in range(nbEpoch):

    model.train()

    for x, y, idx in tqdm(loaderLearning):
        x = x.to(device)
        y = y.to(device)

        out =  model(x)
        

        loss = criterion(out.reshape(-1), y.reshape(-1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if ((j % 100)+1) == 0:
            wandb.log({'epoch': i, 'Training Loss': loss})

        if ((j % 5000)+1) == 0:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for x_val, y_val, _ in loaderValidation:
                    x_val = x_val.to(device)
                    y_val = y_val.to(device)
                    
                    out = model(x_val)
                    
                    val_loss += criterion(out, y_val)
                    
                wandb.log({'epoch': i, 'Validation Loss': val_loss})
            NN.train()

        j += 1

100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


tensor(30.7887, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|██████████| 1/1 [00:02<00:00,  2.17s/it]


tensor(0.0578, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


tensor(0.2652, device='cuda:0', grad_fn=<MseLossBackward0>)


100%|██████████| 1/1 [00:02<00:00,  2.08s/it]

tensor(0.1354, device='cuda:0', grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 